# Description
### 데이터 분석 목표
- 택시 호출 앱 서비스 'iM'의 사용자 인사이트 파악을 위한 사용자 리뷰 데이터 형태소 분석에 앞서,
- 형태소 분석을 위해 필요한 구글 플레이스토어의 리뷰 raw data 크롤링

In [2]:
# 패키지 불러오기
from selenium import webdriver 
from bs4 import BeautifulSoup 
import urllib.request as req 
import pandas as pd 
import numpy as np 
import collections as co 
import time

In [3]:
# 크롤링할 사이트 설정
url = "https://play.google.com/store/apps/details?id=com.mhq.im&showAllReviews=true" 
driverPath = "/Users/michael/Downloads/chromedriver" 
driver = webdriver.Chrome(driverPath) 
driver.get(url)

In [5]:
# 플레이스토어 사이트 데이터 확인을 위한 반복문 설정
SCROLL_PAUSE_TIME = 1.5
last_height = driver.execute_script("return document.body.scrollHeight") 
while True: 
    # (1) 4번의 스크롤링 
    for i in range(5): 
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") 
        time.sleep(SCROLL_PAUSE_TIME) 
    # (2) 더보기 클릭 
    driver.find_element_by_xpath("//span[@class='RveJvd snByac']").click() 
    # (3) 종료 조건 
    new_height = driver.execute_script("return document.body.scrollHeight") 
    if new_height == last_height: 
        break 
    last_height = new_height

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//span[@class='RveJvd snByac']"}
  (Session info: chrome=93.0.4577.63)


In [6]:
# 리뷰 데이터 크롤링
reviews = driver.find_elements_by_xpath("//span[@jsname='bN97Pc']") 
len(reviews)

551

In [9]:
# 리뷰 날짜 데이터 크롤링
dates = driver.find_elements_by_xpath("//span[@class='p2TkOb']") 
len(dates)

1073

In [10]:
# 리뷰 별점 데이터 크롤링
stars = driver.find_elements_by_xpath("//span[@class='nt2C1d']/div[@class='pf5lIe']/div[@role='img']") 
len(stars)

551

In [11]:
# 리뷰 좋아요 데이터 크롤링
likes = driver.find_elements_by_xpath("//div[@aria-label='이 리뷰가 유용하다는 평가를 받은 횟수입니다.']") 
len(likes)

551

In [13]:
# 장문 리뷰와 단문 리뷰 구분
spread_review = driver.find_elements_by_xpath("//button[@jsaction='click:TiglPc']") 
for i in range(len(spread_review)): 
    isTrue = spread_review[i].is_displayed() 
    print("Element is visible? " + str(isTrue)) 
    if isTrue: 
        spread_review[i].click() 
        print(str(i)+"th more button is clicked and wait 1.5 secs...") 
        time.sleep(SCROLL_PAUSE_TIME) 
        
reviews = driver.find_elements_by_xpath("//span[contains(@jsname, 'bN97Pc')]") 
for i in range(len(reviews)): 
    print(str(i) + "\t" + reviews[i].text) 
    
long_reviews = driver.find_elements_by_xpath("//span[@jsname='fbQN7e']") 
for i in range(len(reviews)): 
    print(long_reviews[i].text)

Element is visible? True
0th more button is clicked and wait 1.5 secs...
Element is visible? True
1th more button is clicked and wait 1.5 secs...
Element is visible? True
2th more button is clicked and wait 1.5 secs...
Element is visible? True
3th more button is clicked and wait 1.5 secs...
Element is visible? True
4th more button is clicked and wait 1.5 secs...
Element is visible? True
5th more button is clicked and wait 1.5 secs...
0	쿠폰 준다고 해서 처음 타봣어요 ㅎㅎ 차량이 일단 커서 엄청 좋구요 향기도 좋은 것 같아요 일단 옵션도 말 걸지 않기 조용히 가기? 이런게 잇어서요 되게 세심한 것 같아요 하루죙일 피곤한데 계속 질문하고 말 거시면 피곤하거든요 한 20분 가량 탓는데 정말 편하게 탓어요 계속 이용할 것 같아요 저는 택시도 빨리 잡혓어요ㅋㅋㅋ병원 갓다오고 이럴 때 너무 좋을 것 같네요 근데 제가 택시 할인카드를 쓰는데 여기는 사용내역에 택시라고 이름이 안잡혀서 할인이 안되는 것 같네요 ㅠㅠ그것만 아쉽고 다 너무 좋앗습니다
1	자주타는데 일반 택시랑 가격차이도 많이 안나는데 쾌적하고 기사님들 친절하셔서 너무 좋아요~ 다만 안잡힐땐 너무 안잡히네요ㅠㅠ
2	아가와 함께 이동할 때 최고입니다. 우선 차량이 깨끗하고, 유모차와 아가짐을 실을 만큼 넓어서 좋았어요. 유모차를 실어주신 기사님께도 감사하고요. 다음으로는 쾌적한 차량 내부입니다. 다양한 기능들을 가지고 있었는데 공기청정기라든지 휴대전화 충전기, 패드 등이 인상적이었어요. 마지막으로 배차가 잘 되었습니다. 호출한지 채 몇 분 안되어 곧 도착해주셨습니다. 기존

85	오늘은6월11일새벽에비도많이오고제가짐도많아서IM카니발택시이용했는데오늘완전일반개인택시이용하는기분도들었고완전짜증나고내돈주고이용하는데왜IM카니발택시기사분눈치를봐야하냐요!오늘IM카니발택시기사분완전불친절하게응대해주시고잘못된길로진입하시고나한테짜증을내시고물어보지도않고기사분이잘못된길로들어가놓고나한테왜그러시냐요!저번에는빨래방도착했을때도빨래바구니다안내렸는데카니발차량출입문을닫아서내팔끼어서매우힘들었는데오늘IM카니발택시기사분너무불친절하고매우기분안좋게하시네요!완전Im카니발택시매우실망했어요일반개인택시서비스이용안할려고Im카니발택시이용하는건데도대체왜일반개인택시처...
전체 리뷰
86	마트에서 장보고 워낙 짐이많아서 처음 이용해봤는데 확실히 넓으니 좋긴좋네요ㅎㅎ 짐이 양손가득있어서 문열기도 힘들었는데 문도 자동문이었구 카니발이다보니 짐 싣고도 자리 엄청 넉넉했네요 저처럼 장보고 오시는분들이나 어디 여행가야해서 짐 많은분들이 이용하시기 좋을것 같네요
87	콜택시 였으면 1분 기다릴걸 10분 넘게 기다렸는데 그건 차량이 많지 않겠거니 그렇다쳐요. 성내동에서 마천동 가는데 보통 택시 많이 나와봤자 8000원이에요. 제가 따로 앱에 있는 안전운행 천천히 가달라는 표시를 일부러 체크 안했습니다. 그냥 적당히 빨리 가고싶어서요. 신호는 있는대로 다 걸리고 천천히 처음 가보는 길로 운행하시더니 만원 넘는 요금이 나오네요. 내릴 때 이정도 요금 나오는 거리가 아니라고 했습니다. 서비스 이렇게 하시면 안돼죠. 타다 정도는 되기를 기대했는데.
88	타다플러스 이용하다가 서비스가 점점엉망되서 실망하던중 우연히 알게되서 이용하게 되었는데... 요즘 아임 전파하고 다닙니다. 정말 친절하시고 만족합니다.
89	카니발 택시라 내부가 넓은게 제일 마음에 들었어요ㅋㅋ 이제 곧 여름 오고 장마철 되면 우산 들고 다닐 일이 많아질텐데 짐같은거 많으면 들고 다닐수도 없고 일반 택시는 좁아서 불편했거든요ㅜㅜ 아마 점점 더 자주 이용하지 않을까.. 생각드네요ㅋㅋ
90	저희집이 송파구이다 보니 배차도 1분도 안되서 바로 잡히고

176	와이파이가 되고 태블릿PC를 이용할 수 있는 점이 시대가 진짜 많이 흘렀구나를 알게 해주네요 ㅎㅎㅎ 택시를 오랜만에 타보것도 있긴한데 좋은 경험이었습니다. 일반 택시보다 고급진 내부와 승차감도 좋아서 목적지까지 편하게 잘 이동했습니다~ 감사합니다
177	다 좋은데 지방도 빨리 해주세요
178	아이엠 출퇴근할떄 잘 이용하고 있습니다. 서울 지역에서 이용하는 택시 중에서는 아이엠이 카니발로 승차감이나 내부는 제일 나은 거 같아요~ 저처럼 짐 많이 가지고 다니시는 분들한테는 일반 택시 보다 훨씬 편하긴 합니다. 처음에 보였을 때 호기심에 한번 이용했었는데 그 이후로 계속 찾게 되네요
179	일반 택시만 타다가 친구가 하도 좋다고 하길래 한번 써봤는데 완전 신세계였어요ㅋㅋ 이렇게 내부가 넓은데 공기살균기나 방역도 잘 되있어서 코로나 시국에도 안심하고 탈 수 있는 거 같아요
180	한두번 이용해보니 다른 택시 못타겠어요..ㅋㅋㅋ 처음엔 카니발 택시라 더 비싼거 아닌가 했는데 가격 차이도 없고 차가 크니까 승차감도 좋고 편하기도 하고 다른 분들도 꼭 이용해보시길 추천ㅋㅋㅋ저는 개인적으로 큰차라서 되게 마음에 들었어요 ㅎㅎ
181	마트에서 장보고 오는길에 처음으로 이용해봤는데 카니발 택시라 넓어서 편하게 왔네요~ㅎㅎ 승차감도 생각보다 되게 좋았구요 앞으로 짐많은날엔 아이엠 불러서 이용하려구요~ 별점 5개 드리고가요~~
182	위생적인 면에서 많이 신경쓴 게 많아서 이용하기 좋습니다. 내부에는 공기청정기랑 손소독제가 구비되어 있고, 앞좌석과 뒷좌석 사이에 격벽이 존재해서 안심하고 탔어요. 다음에도 기회가 또 한 번 이용할 생각이에요~
183	택시의 고정관념을 꺠고 카니발로 운영하는 게 참신하고 좋았습니다. 그만큼 차량 내부도 넓어서 편했고 요즘 시기에 맞춰 위생도 깔끔하게 신경 쓰고 있더군요. 만족하면서 잘 이용했습니다. 수고하세요!!
184	앱을 받아서 호출하니까 빠르게 오고 어디로 가고 있는지 보여주니까 괜찮네요 내부엔 푹신하면서 멋진 시트가 있고, 비말 차단기랑

243	처음에 왤케 커 이랬는데 타보니까 확실히 큰게 편하네요 ㅋㅋㅋ 카니발 타고 이동할 줄은 상상도 못했는데 ㅋㅋㅋㅋ 가지고 탄 것도 일때문에 좀 많았는데 기사님이 내려서까지 도와주시더라구요!! 친절에 감동... ㅠㅠ 요즘에 이런 분 찾기가 힘들어서인지 기분 좋더라구용.. 타고 가면서도 승차감 좋고 편안했습니다~ 앞으로 애용할게요!
244	코로나때문에 대중교통도 그렇고 택시도 그렇고 이용할때마다 항상 불안했었는데 아이엠 택시 안에 공기청정기랑 손소독제까지 비치되있어서 간만에 안심하고 이용했네요 택시를 자주 이용하는편은 아니지만 만약 담에 또 택시탈일 있으면 또 부를것 같아요
245	급하게 가야하는데 택시가 안잡혀서 불러봣거든요~ 배차도 바로 뜨고 가까이 계셨는지 빨리 오셔서 잘 타고 시간내에 맞춰서 갔습니다 ㅎㅎㅎ 아슬아슬 했네요
246	친구들이랑 만든 아지트에서 짐 옮길일이 있어서 탔었는데 차가 되게 넓어서 짐 싣고 가기 엄청 좋네요 ㅋㅋㅋ 첨엔 그냥 택시 잡으려고 했는데 친구가 카니발로 가는게 나을 거 같다고 했거든요 ㅋㅋ 아주 굳 초이스였습니다
247	살다살다 카니발 택시를 타는 날도 저에게 오는군요 카니발답게 내부가 넓은 건 기본이고 와이파이라던가 태블릿, 공기청정기 등도 편하게 이용할 수 있게 되어 있더라구요 최근 들어서 회사 야근도 많아져서 저녁마다 피곤한데 자주 이용해야겠어요~!
248	이번에 큰짐을 옮기느라 한번 써봤는데 괜찮네요~ 첫인상이 완전 좋았어요! 우선 짐들고 탈 수 있다는 편안함이랑 넓고 깔끔해서 쾌적했고 기사님도 되게 친절하셔서 흠잡을게 없었네요. 짐이 크면 택시 잡기도 힘들고 한데 다음에도 이런일 있으면 바로 이용할 거 같아요. 짐 아니더라고 유모차나 반려견 있으신 분들도 유용하게 이용하실 수 있을 것 같네욤 필요한일 있으시다면 저는 추천드려요!!
249	여자친구가 이용해보고 좋다길래 저도 한번 이용해봤는데 진짜 좋긴좋네요ㅋㅋㅋ 기사님도 친절하시구 안에서 와 이파이도 터지고 거기다 태블릿 pc까지있음 꽤 먼거리 이동했는데도 덕분에

308	요즘 같은 시국에 차량이 깔끔하니까 아주 만족스럽네요 차량 내부에 공기청정기까지 있는 건 처음타봄.. 생각해보면 요즘에 있는게 맞긴하죠 손님 입장에서도 그렇고 기사님 입장에서도 그렇구요 센스있다고 생각했습니다 ^^
309	지금까지 이용해본 택시 중 가장 깔끔하고 좋았네요 가끔 다른 택시 타다보면 기사님들이 안에서 담배를 피는건지 안좋은 냄새 날때가 많은데 아이엠은 공기청정기가 설치되있어서 그런지 쾌적해서 기분좋게 잘 이용했습니다~
310	코로나가 다시 심해지다 보니까.. 불안해서 택시보다 안정적일거 같아서 불러봤는데 카니발이라 그런지 승차감이 일단 좋네요 ㅎㅎ 태블릿pc도 있고 와이파이까지 사용 가능하고 ㄷㄷ 처음 이용해봐서 약간 신세계 같았어요 ㅋㅋㅋㅋ 공기청정기까지 구비해둔거 보고 방역에도 신경을 많이 쓰는구나 싶어서 더 마음이 놓이고 안심 됐어요 ㅎㅎ
311	카니발이라 그런지 여러명이 탑승해도 될 정도로 넓고 편안해서 좋았어요 안에 들어갈 땐 자동으로 문도 열리고 택시기사 아저씨도 상당히 친철하시구요 목적지까지 편하게 도착했습니다~
312	배송이 빠르고 만족도가 높아요 장거리 사용도 해봤는데 괜찮네요
313	카니발 차량이 커서 그런지 일반 택시보다 훨씬 편하게 느껴졌어요 기사님이 내부도 깔끔해게 해놓으셨고 탔을때 좋은 향기가 나니까 기분도 좋아지더라구요~ 멀리까지는 안갔지만 앞으로 자주 쓸 거 같아요~~
314	택시에서 와이파이 써본건 처음인듯ㅋㅋ 장거리로 이용할 때 데이터 없는 사람들은 와이파이 잡고 심심하지 않게 갈 수 있을 것 같네요ㅋㅋ 저도 단거리보다 장거리를 더 많이 이용해서..
315	카니발 승차감 좋네요 ㅋㅋㅋ 기사님도 친절하시고 처음 이용해보고 전체적으로 만족스러운 서비스였습니다! 마음에 드는 부분이 많아서 앞으로도 자주 이용할 것 같아요
316	비올때 특히 더 좋은 것 같아요ㅋㅋ 일반 택시 타면 우산 접을 때 안젖으려고 몸 꾸겨가면서 타는데 카니발이라 우산 던져놓고 편하게 탈 수 있네요ㅋㅋ 일반 택시 타면 우산에 있는 물기 거의 옷

387	기사님도 친절하시구 승차감도 좋아요 ㅎㅎ 앞으로 자주 이용하겠습니다~~
388	카니발이라 그런지 쾌적한 공간에 승차감도 좋고, 칸막이까지 있어서 코로나 걱정도 덜하네요. 그리고 어플 사용법에 대해서 쉽게 이해할 수 있어서 너무 마음에 들었어요.
389	승차감도 괜찮고 특히 일반택시는 급발진 할 때 많은데 뭔가 더 부드러운 느낌? 거기다 공기살균기기까지 있어서 코로나 걱정은 덜했어요! 외근때문에 이동할 때가 많은데 매번 잘 불러서 쓰고 있습니다~
390	가끔 택시 타면 차 안에서 담배 냄새 나서 좀 불편한 적이 많은데 차 안에 공기청정기도 있고 훨씬 좋은 거 같아요
391	운전기사가 매우 친절해요 기사가 차에서 내려서 손님을 맞이하고 짐도 직접 케어해 줘요 아이엠 짱이예요^^
392	진짜 깨끗하고 친절하고 베스트 택시 입니다! 최신형 카니발도 엣지있어요
393	혹해서 써봤는데 좋네요. 우선 카니반 신형 처음 타봐서 좋았고 기사님 넘친절하시고 내부도 넘 깨끗. 눈길이라 고생하셨는데 끝까지 잘 해주셔서 맘에드네요~
394	기사님이 너무 친절하고 차량의자가 고급이라 편했습니다. 이런 택시라면 자주 이용 할것 같네요.
395	해외로 나갈 일이 생겨서 공항으로 가는데 짐이 너무 많더라고요. 그래서 아이엠 택시 불렀는데 카니발이라 짐이 많아도 충분히 들어가고 좌석이 푹신해서 편안했어요.
396	택시 안에 테블릿 있어서 이동할 때 지루하지 않음 ㅋㅋㅋㅋ
397	편안하고 안전하고 기사님 친절하고 좋은 모빌리티 서비스네요~
398	강남에서 지나가는거 보고 우연히 탔었는데, 너무 좋아서 다음에도 이용하려고 어플도 깔았어요. 어플도 이용하기 좋게 깔끔하게 해놨네요^^
399	너무 좋고요. 기사님이 친절하세요. 짱이네요.
400	다 좋습니다. 근데 실제 도착시 금액 보다 예상 금액이 훨씬 많이 나오던데, 안막혀서 그런가요?
401	편안해서 좋았습니다. 예전 타다 많이 이용 했는데 타다 보다 더 좋은 것 같습니다. 편안하게 목적지까지 잘 왔네요.
402	내부 인테리어 되게 신식

In [ ]:
merged_review = [t.text if t.text!='' else long_reviews[i].text for i,t in enumerate(reviews)] 
likes = driver.find_elements_by_xpath("//div[@aria-label='이 리뷰가 유용하다는 평가를 받은 횟수입니다.']") 
stars = driver.find_elements_by_xpath("//span[@class='nt2C1d']/div[@class='pf5lIe']/div[@role='img']") 
stars_text = stars[3].get_attribute('aria-label')

In [79]:
# 크롤링 데이터 저장
res_deque = co.deque([])
for i in range(len(reviews)):
    res_deque.append({
        'STAR' : stars[i].get_attribute('aria-label'),
        'LIKE' : likes[i].text,
        'REVIEW' : merged_review[i]
    })

res_df = pd.DataFrame(res_deque)
res_df.to_csv('/Users/michael/Downloads/Crawling_data.csv', index=False, encoding='utf-8-sig')